<a href="https://colab.research.google.com/github/chaeun6-cmd/New-repository/blob/main/%EA%B3%A0%EA%B0%9D%EB%B3%84_%EC%BF%A0%ED%8F%B0_%EB%B0%9C%EC%86%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 라이브러리
import pandas as pd
import numpy as np
import datetime as dt
import os

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_excel("/content/Online_Retail.xlsx")

# CSV로 저장
df.to_csv("/content/Online_Retail.xlsx", index=False, encoding="utf-8-sig")

In [4]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
df.dtypes

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
UnitPrice,float64
CustomerID,float64
Country,object


In [6]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [7]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [8]:
# 중복 제거
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

np.int64(0)

In [9]:
#customerID이 없는 행 제거하고 원본 df에 포함
df = df.dropna(subset=["CustomerID"]).copy()
df["CustomerID"] = df["CustomerID"].astype("int64")

#공백처리
df["StockCode"] = df["StockCode"].astype(str)
df["StockCode_clean"] = (df["StockCode"].str.replace(r"\s+", "", regex=True))

#StockCode 알파벳 행 확인
df[df["StockCode_clean"].str.fullmatch(r"[A-Za-z]+", na=False)]["StockCode"].unique()

#StockCode가 알파벳으로만 이루어진 행 제거
"""
- 알파벳으로만 이루어진 행은 고객이 결제한 내역이 아니기 때문
"""
df = df[~df["StockCode_clean"].str.fullmatch(r"[A-Za-z]+", na=False)].copy()

#C2 코드 CARRIAGE 수동 제거
df = df[df["StockCode_clean"] != "C2"].copy()

df.drop(columns=["StockCode_clean"], inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399689 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    399689 non-null  object        
 1   StockCode    399689 non-null  object        
 2   Description  399689 non-null  object        
 3   Quantity     399689 non-null  int64         
 4   InvoiceDate  399689 non-null  datetime64[ns]
 5   UnitPrice    399689 non-null  float64       
 6   CustomerID   399689 non-null  int64         
 7   Country      399689 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 27.4+ MB


#데이터 타입 변환 및 파생변수 생성

In [10]:
# Invoice를 datetime으로 변환
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 시계열 파생변수
# 연도-월
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')

# 시간대
df['Hour'] = df['InvoiceDate'].dt.hour

# 요일 정보 (0=월 1=화 ... 6=일)
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek

# 날짜 추출 (Y-M-D)
df['Date'] = df['InvoiceDate'].dt.date

# 총 판매 금액 변수 생성
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']


"""
- 첫 구매일 변수 생성 : 고객이 우리 서비스를 처음 이용한 날
- 코호트 분석에 사용
- Ex. 2010년 12월에 들어온 고객은 얼마나 남았나?
- 신규 고객 유입 및 유지율 파악
"""
# 각 고객별로 첫 구매일(First Purchase)과 구매 간격(Duration) 계산
customer_first_purchase = df.groupby('CustomerID')['InvoiceDate'].min().reset_index()
customer_first_purchase.columns = ['CustomerID', 'FirstPurchaseDate']

# 원본 데이터에 첫 구매일 병합
df = pd.merge(df, customer_first_purchase, on='CustomerID')

# 고객별 마지막 구매일 계산
last_purchase = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
last_purchase.columns = ['CustomerID', 'LastPurchaseDate']

# 분석 기준일 설정 (데이터의 마지막 날짜 + 1일)
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# 고객별 Recency 계산
last_purchase['Recency_Days'] = (reference_date - last_purchase['LastPurchaseDate']).dt.days

df = df.merge(last_purchase, on='CustomerID', how='left')

print(last_purchase.head())

   CustomerID    LastPurchaseDate  Recency_Days
0       12346 2011-01-18 10:17:00           326
1       12347 2011-12-07 15:52:00             2
2       12348 2011-09-25 13:13:00            75
3       12349 2011-11-21 09:51:00            19
4       12350 2011-02-02 16:01:00           310


In [11]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,Hour,DayOfWeek,Date,TotalAmount,FirstPurchaseDate,LastPurchaseDate,Recency_Days
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12,8,2,2010-12-01,15.30,2010-12-01 08:26:00,2011-02-10 14:38:00,302
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,8,2,2010-12-01,20.34,2010-12-01 08:26:00,2011-02-10 14:38:00,302
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12,8,2,2010-12-01,22.00,2010-12-01 08:26:00,2011-02-10 14:38:00,302
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,8,2,2010-12-01,20.34,2010-12-01 08:26:00,2011-02-10 14:38:00,302
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,8,2,2010-12-01,20.34,2010-12-01 08:26:00,2011-02-10 14:38:00,302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399684,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,2011-12,12,4,2011-12-09,10.20,2011-08-18 15:44:00,2011-12-09 12:50:00,1
399685,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,2011-12,12,4,2011-12-09,12.60,2011-08-18 15:44:00,2011-12-09 12:50:00,1
399686,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,2011-12,12,4,2011-12-09,16.60,2011-08-18 15:44:00,2011-12-09 12:50:00,1
399687,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,2011-12,12,4,2011-12-09,16.60,2011-08-18 15:44:00,2011-12-09 12:50:00,1


#금액 및 상품 정제

In [12]:
# 고객 ID 결측치 제거
df = df[df["CustomerID"].notna()]

# 수량/가격 필터링
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

# 문자열 변환
df["StockCode"] = df["StockCode"].astype(str)
df["Description"] = df["Description"].astype(str)

# 비상품(서비스 항목-배송비, 운송비, 은행 수수료, 온라인 배송비, 샘플 상품) 키워드 정의
service_keywords = ["POST", "CARRIAGE", "BANK", "DOTCOM", "SAMPLE"]
pattern = "|".join(service_keywords)

# 비상품 제거
df = df[
    ~df["StockCode"].str.contains(pattern, case=False, na=False) &
    ~df["Description"].str.contains(pattern, case=False, na=False)
]


# 상품 데이터 정형화(문자열 변환)
df["Description"] = df["Description"].astype(str)

# 대문자 통일
df["Description"] = df["Description"].str.upper()

# 앞뒤 공백 제거
df["Description"] = df["Description"].str.strip()

# 여러 공백을 하나로
df["Description"] = df["Description"].str.replace(r"\s+", " ", regex=True)

# 기준일 생성(최근성을 계산하기 위해)
reference_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

# 가장 최근 구매 날짜/몇 번 주문했는지/총 구매 금액
rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": "max",
    "InvoiceNo": "nunique",
    "TotalAmount": "sum"
}).reset_index()

# 생성된 기준일을 기준으로 계산
rfm["Recency"] = (reference_date - rfm["InvoiceDate"]).dt.days

# 컬럼 이름 변환
rfm = rfm.rename(columns={
    "InvoiceNo": "Frequency",
    "TotalAmount": "Monetary"
})

rfm.head()

,CustomerID,InvoiceDate,Frequency,Monetary,Recency
0,12346,2011-01-18 10:01:00,1,77183.60,326
1,12347,2011-12-07 15:52:00,7,4310.00,2
2,12348,2011-09-25 13:13:00,4,1437.24,75
3,12349,2011-11-21 09:51:00,1,1457.55,19
4,12350,2011-02-02 16:01:00,1,294.40,310


In [13]:
rfm.tail()

,CustomerID,InvoiceDate,Frequency,Monetary,Recency
4329,18280,2011-03-07 09:52:00,1,180.60,278
4330,18281,2011-06-12 10:53:00,1,80.82,181
4331,18282,2011-12-02 11:43:00,2,178.05,8
4332,18283,2011-12-06 12:02:00,16,2035.23,4
4333,18287,2011-10-28 09:29:00,3,1819.88,43


#검증 및 요약

In [14]:
invoice_df = df.groupby(
    ["CustomerID", "InvoiceNo"]
).agg(
    OrderDate=("InvoiceDate", "min"),
    OrderAmount=("TotalAmount", "sum")
).reset_index()

# 기준 날짜 설정
reference_date = invoice_df["OrderDate"].max() + dt.timedelta(days=1)

# RFM 생성
rfm = invoice_df.groupby("CustomerID").agg(
    Recency=("OrderDate", lambda x: (reference_date - x.max()).days),
    Frequency=("InvoiceNo", "nunique"),
    Monetary=("OrderAmount", "sum")
).reset_index()

In [15]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,12346,326,1,77183.60
1,12347,2,7,4310.00
2,12348,75,4,1437.24
3,12349,19,1,1457.55
4,12350,310,1,294.40


In [16]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,Hour,DayOfWeek,Date,TotalAmount,FirstPurchaseDate,LastPurchaseDate,Recency_Days
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12,8,2,2010-12-01,15.30,2010-12-01 08:26:00,2011-02-10 14:38:00,302
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,8,2,2010-12-01,20.34,2010-12-01 08:26:00,2011-02-10 14:38:00,302
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12,8,2,2010-12-01,22.00,2010-12-01 08:26:00,2011-02-10 14:38:00,302
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,8,2,2010-12-01,20.34,2010-12-01 08:26:00,2011-02-10 14:38:00,302
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,8,2,2010-12-01,20.34,2010-12-01 08:26:00,2011-02-10 14:38:00,302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399684,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,2011-12,12,4,2011-12-09,10.20,2011-08-18 15:44:00,2011-12-09 12:50:00,1
399685,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,2011-12,12,4,2011-12-09,12.60,2011-08-18 15:44:00,2011-12-09 12:50:00,1
399686,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,2011-12,12,4,2011-12-09,16.60,2011-08-18 15:44:00,2011-12-09 12:50:00,1
399687,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,2011-12,12,4,2011-12-09,16.60,2011-08-18 15:44:00,2011-12-09 12:50:00,1


In [22]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R_rank,F_rank,M_rank,R_score,F_score,M_score,RFM_score
0,12346,326,1,77183.60,4170.0,2829.0,9.0,1,2,5,8
1,12347,2,7,4310.00,94.0,554.0,324.0,5,5,5,15
2,12348,75,4,1437.24,2672.0,1102.0,1212.0,2,4,4,10
3,12349,19,1,1457.55,1125.0,2830.0,1196.0,4,2,4,10
4,12350,310,1,294.40,4100.0,2831.0,3291.0,1,2,2,5


In [23]:
# 고객 단위 집계
rfm = df.groupby('CustomerID').agg({
    'Recency_Days': 'min',
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum',
    'FirstPurchaseDate': 'min'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary', 'FirstPurchaseDate']

In [24]:
# 5점 척도로 나눔
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

rfm['RFM_score'] = rfm[['R_score','F_score','M_score']].sum(axis=1)

In [25]:
# 분석 기준일(InvoiceDate)로 잡고 최근 30일 이내 첫 구매 -> 신규 고객
analysis_date = df['InvoiceDate'].max()

rfm['NewCustomer'] = (
    (analysis_date - rfm['FirstPurchaseDate']).dt.days <= 30
)

In [26]:
# VIP/일반 고객 나누기
rfm['percentile'] = rfm['RFM_score'].rank(pct=True)

rfm['Segment'] = 'General'
rfm.loc[rfm['percentile'] >= 0.8, 'Segment'] = 'VIP'
rfm.loc[rfm['NewCustomer'] == True, 'Segment'] = 'New'

In [31]:
# 세그먼트별 지표 비교
segment_summary = rfm.groupby('Segment').agg({
    'Recency':'mean',
    'Frequency':'mean',
    'Monetary':'mean',
    'CustomerID':'count'
}).reset_index()

# 세그먼트별 지표 확인
rfm['Segment'].value_counts()

,count
Segment,
General,3138
VIP,922
New,274


#고객별 평균 구매 간격 계산

In [32]:
# 거래 데이터 정렬
df = df.sort_values(['CustomerID','InvoiceDate'])

# 이전 구매일
df['PrevDate'] = df.groupby('CustomerID')['InvoiceDate'].shift(1)

# 구매 간격(일)
df['Gap'] = (df['InvoiceDate'] - df['PrevDate']).dt.days

# 고객별 평균 구매 간격
gap_df = df.groupby('CustomerID')['Gap'].mean().reset_index()
gap_df.columns = ['CustomerID','AvgPurchaseGap']

In [33]:
# rfm
rfm = rfm.merge(gap_df, on='CustomerID', how='left')

In [34]:
# 세그먼트별 구매주기 / 최신 비교
rfm.groupby('Segment').agg({
    'Recency':'mean',
    'AvgPurchaseGap':'mean'
}).reset_index()

,Segment,Recency,AvgPurchaseGap
0,General,120.036648,3.701631
1,New,16.321168,0.096081
2,VIP,13.545553,2.688571


**VIP, 일반고객, 신규 -> 구매주기 초과 (쿠폰 필요 가능성이 높음)**

In [35]:
# 구매주기 초과 여부
rfm['OverCycle'] = rfm['Recency'] > rfm['AvgPurchaseGap']

rfm.groupby('Segment')['OverCycle'].mean().reset_index()

,Segment,OverCycle
0,General,0.942957
1,New,0.963504
2,VIP,0.875271


**신규 > 일반 고객 > VIP 순으로 쿠폰 타겟 적합**

In [38]:
# 세그먼트별 재구매율 확인
rfm['SecondPurchase'] = rfm['Frequency'] >= 2

rfm.groupby('Segment')['SecondPurchase'].mean().reset_index()

,Segment,SecondPurchase
0,General,0.591460
1,New,0.182482
2,VIP,1.000000


#1️⃣ VIP → 100%
- 이미 2회 이상 구매한 고객만 포함
- 재구매율 100%
- 쿠폰 필요성 매우 낮음
- 마진만 깍일 가능성이 높음
-> 쿠폰 타켓이 아님

#2️⃣ General → 59%
- 절반 이상은 자연 재구매
- 41% 재구매 안 함
-> 구매 주기 초과 고객(일부)만 쿠폰 타겟팅

#3️⃣ New → 18%
- 신규 고객 중 81%가 재구매 안 함
- 1회성 구매 고객이 많음
- 재구매 전환이 매우 낮음
-> 재구매 유도 쿠폰이 가장 효과적으로 나타날 수 있음

In [39]:
# 세그먼트별 전체 매출 중 비중 확인(매출 기여도)
segment_revenue = rfm.groupby('Segment')['Monetary'].sum().reset_index()

segment_revenue['RevenueShare'] = (
    segment_revenue['Monetary'] / segment_revenue['Monetary'].sum()
)

segment_revenue

,Segment,Monetary,RevenueShare
0,General,2502294.06,0.287323
1,New,123938.50,0.014231
2,VIP,6082762.77,0.698446


**1. VIP** - 총 매출 약 70% 차지 (핵심 매출원)

**2. 일반 고객** - 총 매출의 약 29% 차지

**3. 신규** - 총 매출의 1.42% 차지

- 시즌성
- 프로모션
- 외부 이벤트
-> 영향으로 객관성이 조금 떨어질 수도 있음